In [14]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
import random
from colorama import Fore, Style, init
init(autoreset=True)

### 1. Degree Anonymization (Dynamic Programming & Greedy)

In [15]:
import numpy as np

class DegreeAnonymizer:
    def __init__(self, degrees, k):
        # Store the original degrees and their sorted order with indices
        self.original_degrees = degrees.copy()
        self.sorted_nodes = sorted(range(len(degrees)), key=lambda x: degrees[x], reverse=True)
        self.sorted_degrees = [degrees[i] for i in self.sorted_nodes]
        self.n = len(degrees)
        self.k = k

    def dynamic_programming(self):
        n = self.n
        k = self.k
        dp = [float('inf')] * (n + 1)  # dp[i] = minimal cost for the first i nodes
        dp[0] = 0
        I = self._precompute_I()
        best_t = [-1] * (n + 1)  # Track the best split points for reconstruction

        # Dynamic programming to compute minimal anonymization cost
        for i in range(1, n + 1):
            if i < k:
                dp[i] = I[0][i-1]
                best_t[i] = 0  # Group [0, i-1]
            else:
                start = max(k, i - 2*k + 1)
                end = i - k + 1
                if start < end:
                    min_cost = float('inf')
                    best_split = start
                    for t in range(start, end):
                        current_cost = dp[t] + I[t][i-1]
                        if current_cost < min_cost:
                            min_cost = current_cost
                            best_split = t
                    dp[i] = min(dp[i], min_cost)
                    best_t[i] = best_split
                else:
                    dp[i] = I[0][i-1]
                    best_t[i] = 0  # Fallback to grouping all nodes [0, i-1]

        # Reconstruct the anonymized sequence using best_t
        anonymized = [0] * n  # Preallocate anonymized sequence
        current = n
        while current > 0:
            t = best_t[current]
            group_start = t
            group_end = current - 1  # Because current corresponds to i in dp[i]
            assigned_degree = max(self.sorted_degrees[group_start:group_end+1])  # Maximum degree in the group
            
            # Map anonymized degrees back to original node indices
            for idx in range(group_start, group_end + 1):
                original_node = self.sorted_nodes[idx]
                anonymized[original_node] = assigned_degree
            current = t

        return anonymized

    def greedy(self):
        groups = []
        i = 0
        while i < self.n:
            current_degree = self.sorted_degrees[i]
            j = i

            # Expand the group until it has at least k elements
            while (j + 1 < self.n and (j - i + 1) < self.k):
                j += 1

            # Enforce 2k-1 group size restriction explicitly
            j = min(j, self.n - 1)
            if (j - i + 1) > 2 * self.k - 1:
                j = i + 2 * self.k - 2

            # Set all degrees in this group to the maximum degree in the group
            max_deg = self.sorted_degrees[i]
            groups.append((i, j, max_deg))
            i = j + 1

        # Construct the anonymized list clearly and safely
        anonymized = [0] * self.n  # Preallocate the anonymized list
        for start, end, deg in groups:
            for idx in range(start, end + 1):
                original_node = self.sorted_nodes[idx]
                anonymized[original_node] = deg

        return anonymized

    def _precompute_I(self):
        I = np.full((self.n, self.n), np.inf)
        for i in range(self.n):
            for j in range(i, self.n):
                group_size = j - i + 1
                if self.k <= group_size <= 2 * self.k - 1:  # Enforce group size restriction
                    max_degree = self.sorted_degrees[i]
                    cost = sum(max_degree - self.sorted_degrees[x] for x in range(i, j+1))
                    I[i][j] = cost
        return I

In [16]:
# Generate a random original graph (Barabasi-Albert model)
k = 3  # Ensure at least k-anonymity
num_nodes = 20
# edges_per_new_node = 5
# original_graph = nx.barabasi_albert_graph(num_nodes, edges_per_new_node)

p = 0.3  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)


# Extract the degree sequence of the original graph
original_degrees = sorted([d for _, d in original_graph.degree()], reverse=True)

# Initialize the DegreeAnonymizer
degree_anonymizer = DegreeAnonymizer(original_degrees, k)

# Perform degree anonymization using Dynamic Programming
anonymized_degrees_dp = degree_anonymizer.dynamic_programming()

# Perform degree anonymization using Greedy
anonymized_degrees_greedy = degree_anonymizer.greedy()

# Create DataFrames to display results
df_large_dp = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (DP)": anonymized_degrees_dp
})

df_large_greedy = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (Greedy)": anonymized_degrees_greedy
})

# Display results
print("Dynamic Programming Results:")
print(df_large_dp)

print("\nGreedy Results:")
print(df_large_greedy)

Dynamic Programming Results:
    Original Degrees  Anonymized Degrees (DP)
0                 10                       10
1                  8                       10
2                  7                       10
3                  7                        7
4                  6                        7
5                  6                        7
6                  6                        6
7                  6                        6
8                  6                        6
9                  6                        6
10                 4                        4
11                 4                        4
12                 4                        4
13                 4                        4
14                 4                        4
15                 4                        4
16                 3                        3
17                 3                        3
18                 2                        3
19                 2                        3

Gree

### 2. Graph Construction (Supergraph Algorithm)

In [17]:
class GraphConstructor:
    def __init__(self, G, anonymized_degrees):
        """
        Initializes the GraphConstructor with a graph and anonymized degree sequence.
        - Converts node labels to integers for consistency.
        - Stores the anonymized degree sequence and the number of nodes.
        """
        self.G = nx.convert_node_labels_to_integers(G, ordering='sorted')  # Ensure nodes are labeled as integers
        self.anonymized_degrees = anonymized_degrees
        self.n = len(anonymized_degrees)  # Number of nodes in the graph
        self.original_edges = set(self.G.edges())  # Store original edges

    def supergraph(self):
        """
        Constructs a supergraph by adding edges to satisfy residual degrees.
        Ensures that no duplicate edges or self-loops are created.
        """
        print(Fore.CYAN + "[DEBUG] Supergraph construction started.")

        # Step 1: Initialize edge set and compute residual degrees
        edges = set(self.original_edges)  # Start with the original edges
        residual = [self.anonymized_degrees[i] - self.G.degree(i) for i in range(self.n)]  # Compute initial residuals

        print(Fore.YELLOW + f"[DEBUG] Initial residual degrees: {residual}")

        iteration = 0  # Track the number of iterations

        # Step 2: Iteratively add edges until all residuals are resolved
        while sum(residual) > 0:
            iteration += 1
            print(Fore.CYAN + f"[DEBUG] Supergraph iteration {iteration}, total residual sum: {sum(residual)}")

            # Identify nodes with positive residuals (nodes that still need edges)
            candidates = sorted([i for i in range(self.n) if residual[i] > 0], key=lambda x: -residual[x])
            progress = False  # Track whether any progress was made in this iteration

            # Iterate over all pairs of candidate nodes
            for u in candidates:
                if residual[u] <= 0:  # Skip nodes whose residuals are already resolved
                    continue

                for v in candidates:
                    if u == v:  # Avoid self-loops
                        continue

                    # Ensure the edge is valid: no duplicates or existing edges
                    if (u, v) not in edges and (v, u) not in edges and residual[v] > 0:
                        edges.add((u, v))  # Add the edge to the graph
                        residual[u] -= 1  # Decrease residuals for both nodes
                        residual[v] -= 1
                        progress = True

                        print(Fore.GREEN + f"[DEBUG] Added edge ({u}, {v}). "
                              f"Residual now: node {u}: {residual[u]}, node {v}: {residual[v]}")

                        if residual[u] == 0 or residual[v] == 0:  # Stop if residuals are resolved
                            break

            # Check if progress was made in this iteration
            if not progress:
                print(Fore.RED + "[ERROR] No further edge additions possible. Breaking loop to prevent infinite execution.")
                break

        # Step 3: Construct the final graph
        constructed_graph = nx.Graph()
        constructed_graph.add_nodes_from(range(self.n))  # Add all nodes
        constructed_graph.add_edges_from(edges)  # Add all edges

        # Verify the constructed graph's degree sequence
        constructed_degrees = [deg for _, deg in constructed_graph.degree()]
        print(Fore.GREEN + "[DEBUG] Successfully constructed supergraph.")
        print(Fore.GREEN + f"[DEBUG] Constructed degrees: {sorted(constructed_degrees, reverse=True)}")
        print(Fore.GREEN + f"[DEBUG] Expected anonymized degrees: {sorted(self.anonymized_degrees, reverse=True)}")

        return constructed_graph

### 3. Probing Scheme

In [18]:
def probing_scheme(G, k, anonymizer_func, max_iter=100):
    G = nx.convert_node_labels_to_integers(G)
    original_degrees = [d for _, d in G.degree()]
    n = len(original_degrees)
    
    print(Fore.CYAN + "\n--- Step 0: Initialization ---")
    print(Fore.YELLOW + f"[DEBUG] Original graph has {n} nodes.")
    print(Fore.YELLOW + f"[DEBUG] Original degrees (node: degree): { {i: original_degrees[i] for i in range(n)} }")

    modified_nodes = set()
    best_G_anon = None
    best_edge_overlap = 0.0

    current_degrees = original_degrees.copy()  # <-- persistently updated degrees!

    for iteration in range(max_iter):
        print(Fore.CYAN + f"\n--- Iteration {iteration + 1}/{max_iter} ---")
        print(Fore.YELLOW + f"[DEBUG] Current original degrees: {current_degrees}")

        # Step 1: Anonymization
        anonymizer = DegreeAnonymizer(current_degrees, k)
        anonymized_degrees = anonymizer_func(anonymizer)
        print(Fore.GREEN + "[DEBUG] Step 1: Anonymization Result:")
        print(Fore.GREEN + f"[DEBUG] Anonymized degrees: {anonymized_degrees}")

        # Step 2: Graphicality check
        is_graphical_flag = is_graphical(anonymized_degrees)
        print(Fore.MAGENTA + f"[DEBUG] Step 2: Graphicality check result: {is_graphical_flag}")

        if not is_graphical_flag:
            print(Fore.RED + "[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.")
            
            # Adjust the lowest-degree node that hasn't been modified yet
            sorted_nodes = sorted(range(n), key=lambda x: (current_degrees[x], x in modified_nodes))
            node_to_adjust = next((node for node in sorted_nodes if node not in modified_nodes), None)
            
            if node_to_adjust is None:
                print(Fore.RED + "[DEBUG] All nodes modified. Resetting modified_nodes set.")
                modified_nodes.clear()
                node_to_adjust = sorted_nodes[0]
            
            old_degree = current_degrees[node_to_adjust]
            new_degree = min(old_degree + 1, n - 1)
            current_degrees[node_to_adjust] = new_degree
            modified_nodes.add(node_to_adjust)

            print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
            print(Fore.RED + f"[DEBUG] Modified nodes: {modified_nodes}")
            continue # Because the current anonymized degree sequence failed the graphicality check

        # Step 4: Graph construction
        print(Fore.GREEN + "[DEBUG] Step 4: Graph construction attempt.")
        constructor = GraphConstructor(G, anonymized_degrees)
        G_anon = constructor.supergraph()

        if G_anon is not None:
            print(Fore.GREEN + "[DEBUG] Graph construction SUCCEEDED.")
            
            # Step 5: Greedy_Swap optimization
            print(Fore.BLUE + "[DEBUG] Step 5: Greedy_Swap optimization started.")
            swapper = GreedySwap(G, G_anon)
            G_anon_optimized = swapper.swap()
            print(Fore.BLUE + "[DEBUG] Greedy_Swap optimization completed.")

            # Edge overlap
            original_edges = set(G.edges())
            anonymized_edges = set(G_anon_optimized.edges())
            edge_overlap = len(original_edges & anonymized_edges) / len(original_edges)
            print(Fore.BLUE + f"[DEBUG] Edge overlap ratio: {edge_overlap:.2f}")

            return G_anon_optimized

        # If construction failed:
        print(Fore.RED + "[DEBUG] Graph construction FAILED. Adjusting degrees again.")
        sorted_nodes = sorted(range(n), key=lambda x: (x in modified_nodes, current_degrees[x]))
        node_to_adjust = sorted_nodes[0]
        old_degree = current_degrees[node_to_adjust]
        new_degree = min(old_degree + 1, n - 1)
        current_degrees[node_to_adjust] = new_degree
        modified_nodes.add(node_to_adjust)

        print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
        print(Fore.RED + f"[DEBUG] Updated original degrees: {current_degrees}")

    print(Fore.RED + "[DEBUG] --- Probing scheme FAILED after maximum iterations ---")
    print(Fore.RED + "[DEBUG] Falling back to the best anonymized graph found.")
    return best_G_anon if best_G_anon is not None else nx.complete_graph(n)


In [19]:
def validate_k_anonymity(d_anon, k):
    groups = {}
    for degree in d_anon:
        groups[degree] = groups.get(degree, 0) + 1 # get is used for counting how many times each degree appears in d_anon
    for count in groups.values():
        if count < k:
            return False
    return True

In [20]:
def evaluate_edge_overlap(G_original, G_anon):
    original_edges = set(G_original.edges())
    anonymized_edges = set(G_anon.edges())
    edge_intersect = len(original_edges & anonymized_edges) / len(original_edges)
    return edge_intersect

In [21]:
def is_graphical(degrees):
    """
    Check if a degree sequence is graphical using the Erdős–Gallai theorem.

    Parameters:
        degrees (list): A list of integers representing the degree sequence.

    Returns:
        bool: True if the sequence is graphical, False otherwise.
    """
    # Step 1: Sort the degree sequence in non-increasing order
    degrees = sorted(degrees, reverse=True)
    n = len(degrees)

    # Step 2: Check if the sum of degrees is even
    total_degrees = sum(degrees)
    if total_degrees % 2 != 0:
        return False  # The sum of degrees must be even for a graph to exist

    # Step 3: Apply the Erdős–Gallai condition
    for k in range(1, n + 1):
        sum_deg = sum(degrees[:k])  # Sum of the first k degrees
        max_sum = k * (k - 1) + sum(min(d, k) for d in degrees[k:])  # Maximum possible sum
        if sum_deg > max_sum:
            return False  # Erdős–Gallai condition violated

    return True  # The sequence is graphical

In [22]:
# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 20 nodes.
[DEBUG] Original degrees (node: degree): {0: 4, 1: 6, 2: 3, 3: 4, 4: 4, 5: 3, 6: 10, 7: 6, 8: 2, 9: 4, 10: 6, 11: 8, 12: 4, 13: 7, 14: 6, 15: 2, 16: 6, 17: 6, 18: 4, 19: 7}

--- Iteration 1/100 ---
[DEBUG] Current original degrees: [4, 6, 3, 4, 4, 3, 10, 6, 2, 4, 6, 8, 4, 7, 6, 2, 6, 6, 4, 7]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [4, 7, 3, 4, 4, 3, 10, 7, 3, 4, 6, 10, 4, 10, 6, 3, 6, 6, 4, 7]
[DEBUG] Step 2: Graphicality check result: False
[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.
[DEBUG] Adjusted node 8: degree 2 → 3
[DEBUG] Modified nodes: {8}

--- Iteration 2/100 ---
[DEBUG] Current original degrees: [4, 6, 3, 4, 4, 3, 10, 6, 3, 4, 6, 8, 4, 7, 6, 2, 6, 6, 4, 7]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [4, 7, 3, 4, 4, 3, 10, 7, 3, 4, 6, 10, 4, 10, 6, 3, 6, 6, 4, 7]
[DEBUG] Step 2: Graphicality check result: False
[DEBUG] Step 3: Graphicality FAILED,

### 3. Greedy_Swap Algorithm

In [23]:
class GreedySwap:
    def __init__(self, G_original, G_anon):
        self.G_original = G_original
        self.G_anon = G_anon
        self.original_edges = set(G_original.edges())
        self.common_edges = set(G_original.edges()) & set(G_anon.edges())
        print(Fore.YELLOW + f"[DEBUG] Initial common edges count: {len(self.common_edges)}")

    def find_max_swap(self):
        max_gain = -float('inf')  # Allow swaps even if gain is 0
        best_swap = None
        edges = list(self.G_anon.edges())

        # Sample edges for efficiency
        sampled_edges = random.sample(edges, min(len(edges), int(len(edges) ** 0.5)))

        for i in range(len(sampled_edges)):
            for j in range(i + 1, len(sampled_edges)):
                u1, v1 = sampled_edges[i]
                u2, v2 = sampled_edges[j]

                if len({u1, v1, u2, v2}) < 4:
                    continue

                swap_options = [
                    ((u1, u2), (v1, v2)),  # Swap type 1
                    ((u1, v2), (v1, u2))   # Swap type 2
                ]

                for new_e1, new_e2 in swap_options:
                    if self.G_anon.has_edge(*new_e1) or self.G_anon.has_edge(*new_e2):
                        continue

                    gain = 0
                    # Reward swaps that introduce new original edges
                    if new_e1 in self.original_edges:
                        gain += 2
                    if new_e2 in self.original_edges:
                        gain += 2
                    # Penalize removing original edges
                    if (u1, v1) in self.common_edges:
                        gain -= 1
                    if (u2, v2) in self.common_edges:
                        gain -= 1

                    # ✅ **Introduce randomness to escape local minima**
                    gain += random.uniform(-0.5, 0.5)  

                    # ✅ Allow swaps with gain ≥ -1 to introduce more variance
                    if gain >= max_gain:
                        max_gain = gain
                        best_swap = (
                            (u1, v1),   # Edge 1 to REMOVE 
                            (u2, v2),   # Edge 2 to REMOVE
                            new_e1,     # Edge 1 to ADD
                            new_e2      # Edge 2 to ADD
                        )

        print(Fore.CYAN + f"[DEBUG] find_max_swap completed. Best gain: {max_gain}, Best swap: {best_swap}")
        return max_gain, best_swap  # Allow swaps with gain ≥ -1


    def swap(self, max_iter=100, force_swap_every=3):
        swapped_edges = set()  # Track already swapped edges

        for iteration in range(max_iter):
            print(Fore.CYAN + f"[DEBUG] Swap iteration {iteration + 1}")
            gain, swap_edges = self.find_max_swap()

            # 🔹 **If no beneficial swaps, force a random swap every few iterations**
            if swap_edges is None or gain < -1:
                if iteration % force_swap_every == 0:
                    print(Fore.RED + "[DEBUG] No beneficial swap found. Forcing a random structural swap.")
                    edges = list(self.G_anon.edges())
                    if len(edges) > 2:
                        e1, e2 = random.sample(edges, 2) # Pick 2 random edges
                        u1, v1 = e1 # Unpack first edge (u1, v1)
                        u2, v2 = e2 # Unpack second edge (u2, v2)
                        new_e1, new_e2 = (u1, u2), (v1, v2) # Create new edges by reconnecting nodes - pay attention to u1,u2 and v1,v2
                        if not self.G_anon.has_edge(*new_e1) and not self.G_anon.has_edge(*new_e2):
                            swap_edges = (e1, e2, new_e1, new_e2)
                else:
                    print(Fore.RED + "[DEBUG] No beneficial swap found. Ending optimization.")
                    break

            e1, e2, new_e1, new_e2 = swap_edges

            # Prevent repeatedly swapping the same edges
            if (e1 in swapped_edges or e2 in swapped_edges) and iteration > 5: # Is used to prevent the function from skipping too many swaps early in the process.
                print(Fore.RED + f"[DEBUG] Skipping redundant swap: {swap_edges}")
                continue

            swapped_edges.add(e1)
            swapped_edges.add(e2)

            print(Fore.BLUE + f"[DEBUG] Performing swap: remove {e1}, {e2}; add {new_e1}, {new_e2}")

            self.G_anon.remove_edges_from([e1, e2])
            self.G_anon.add_edges_from([new_e1, new_e2])

            # Update common edges
            self.common_edges.discard(e1)
            self.common_edges.discard(e2)
            if new_e1 in self.original_edges:
                self.common_edges.add(new_e1)
            if new_e2 in self.original_edges:
                self.common_edges.add(new_e2)

        print(Fore.YELLOW + f"[DEBUG] Common edges count after swap: {len(self.common_edges)}")


        return self.G_anon


### 5. Evaluation Metrics

In [24]:
def evaluate(G_original, G_anon):
    metrics = {}
    
    # L1 norm of degree differences
    l1 = sum(abs(G_original.degree(n) - G_anon.degree(n)) for n in G_original.nodes())
    metrics['L1'] = l1
    
    # Clustering Coefficient
    cc_orig = nx.average_clustering(G_original)
    cc_anon = nx.average_clustering(G_anon)
    metrics['CC_original'] = cc_orig
    metrics['CC_anon'] = cc_anon
    
    # Average Path Length (largest connected component)
    def largest_connected_component_apl(G):
        largest_cc = max(nx.connected_components(G), key=len)
        return nx.average_shortest_path_length(G.subgraph(largest_cc))
    
    apl_orig = largest_connected_component_apl(G_original)
    apl_anon = largest_connected_component_apl(G_anon)
    metrics['APL_original'] = apl_orig
    metrics['APL_anon'] = apl_anon
    
    # Edge Intersection
    if len(G_original.edges()) == 0:
        edge_intersect = 0
    else:
        edge_intersect = len(set(G_original.edges()) & set(G_anon.edges())) / len(G_original.edges())
    metrics['Edge_Intersection'] = edge_intersect
    
    return metrics

### 6. Usage

In [26]:
# Generate a random original graph (Barabasi-Albert model)
k = 5  # Ensure at least k-anonymity
num_nodes = 30

# k = 3  # Ensure at least k-anonymity
# num_nodes = 200

p = 0.2  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)

# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")

# Evaluate results for Greedy
if G_anon_greedy is not None:
    metrics_greedy = evaluate(original_graph, G_anon_greedy)
    print("\nEvaluation Metrics (Greedy):", metrics_greedy)
else:
    print("\nGreedy Anonymization failed.")

# Evaluate results for Dynamic Programming
if G_anon_dp is not None:
    metrics_dp = evaluate(original_graph, G_anon_dp)
    print("\nEvaluation Metrics (Dynamic Programming):", metrics_dp)
else:
    print("\nDynamic Programming Anonymization failed.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 30 nodes.
[DEBUG] Original degrees (node: degree): {0: 7, 1: 5, 2: 7, 3: 7, 4: 6, 5: 5, 6: 6, 7: 1, 8: 4, 9: 6, 10: 6, 11: 5, 12: 9, 13: 3, 14: 6, 15: 5, 16: 8, 17: 8, 18: 6, 19: 5, 20: 6, 21: 8, 22: 7, 23: 8, 24: 6, 25: 5, 26: 5, 27: 13, 28: 7, 29: 6}

--- Iteration 1/100 ---
[DEBUG] Current original degrees: [7, 5, 7, 7, 6, 5, 6, 1, 4, 6, 6, 5, 9, 3, 6, 5, 8, 8, 6, 5, 6, 8, 7, 8, 6, 5, 5, 13, 7, 6]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees: [8, 5, 8, 8, 6, 5, 6, 5, 5, 6, 6, 5, 13, 5, 6, 5, 13, 13, 6, 5, 6, 13, 8, 8, 6, 5, 5, 13, 8, 6]
[DEBUG] Step 2: Graphicality check result: False
[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.
[DEBUG] Adjusted node 7: degree 1 → 2
[DEBUG] Modified nodes: {7}

--- Iteration 2/100 ---
[DEBUG] Current original degrees: [7, 5, 7, 7, 6, 5, 6, 2, 4, 6, 6, 5, 9, 3, 6, 5, 8, 8, 6, 5, 6, 8, 7, 8, 6, 5, 5, 13, 7, 6]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymi